In [1]:
import geojson
from glob import glob
import json
import os, sys, fnmatch
from os import makedirs, path as op
import shutil
from PIL import Image
import numpy as np

In [2]:
def find(pattern, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, pattern):
                result.append(os.path.join(root, name))
    return result

In [4]:
def get_bounding_box(geoj):
    with open(geoj) as f:
        xcoords = []
        ycoords = []
        data = json.load(f)
        for feature in data['features']:
            geom = feature['geometry']
            for coord in geom['coordinates']:
                if type(coord) == float:  # then its a point feature
                    xcoords.append(geom['coordinates'][0])
                    ycoords.append(geom['coordinates'][1])
                elif type(coord) == list:
                    for c in coord:
                        if type(c) == float:  # then its a linestring feature
                            xcoords.append(coord[0])
                            ycoords.append(coord[1])
                        elif type(c) == list:  # then its a polygon feature
                            xcoords.append(c[0])
                            ycoords.append(c[1])
            coords = np.array(list(geojson.utils.coords(geom)))
    return [min(xcoords), min(ycoords), max(xcoords), max(ycoords)]

In [5]:
root_dir = ''

In [ ]:
order_id = ''

In [24]:
imagery_dir = f'{root_dir}/imagery/data/{order_id}/PSScene3Band'

In [25]:
scene_ids = []

In [ ]:
geo_dir = f'{root_dir}/geojsons'

In [37]:
config = f'{root_dir}/config.json'

In [38]:
def make_config_labelmaker(config, scene):
    geoj = f'{geo_dir}/{scene}.geojson'
    bbox = get_bounding_box(geoj)
    with open(config, 'r') as con_j:
        config_json = json.load(con_j)
        config_json['bounding_box'] = bbox
        config_json['geojson'] = geoj
        config_json['imagery'] = f'{imagery_dir}/{scene}_3B_Visual.tif'
    if not os.path.isdir(f'{root_dir}/configs/{scene}'):
        os.makedirs(f'{root_dir}/configs/{scene}')
    config_scene = f'{root_dir}/configs/{scene}/config.json'
    with open(config_scene, 'w') as con_n_j:
        json.dump(config_json, con_n_j)
        print(config_json)
    print(f"Processed {scene}")

In [ ]:
for scene in scene_ids:
    make_config_labelmaker(config, scene)

In [42]:
# run ./batch_lblmkr.sh from root_dir

In [35]:
def tif2jpg(scene):
    os.chdir(f'{root_dir}/configs/{scene}')
    tifs = glob(f'data/tiles'+'/**/*.tif',recursive=True)
    for i in tifs:
        im = Image.open(i)
        im.save(i[:-4]+'.jpg', "JPEG")
    for f in tifs:
        os.remove(f)                
    os.chdir(f'{root_dir}')
    print(f"Processed {scene}")

In [ ]:
for scene in scene_ids:
    tif2jpg(scene)